In [2]:
import os
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torchvision.models import alexnet

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
assert use_cuda == True
device

device(type='cuda', index=0)

In [3]:
from pylab import *
from IPython.core.debugger import set_trace

%matplotlib inline

In [4]:
"""
Loads Alex NN in gpu
Sets optimizer and Loss function
"""
#
a_model = alexnet()
#a_model = torch.load('AlexMnist')
a_model = a_model.to(device)
#a_model = resnet18().cuda()
#n_ftrs=a_model.fc.in_features
#a_model.fc=torch.nn.Linear(n_ftrs,10)
optimizer = optim.SGD(a_model.parameters(), lr=0.01, momentum=0.9)
kernel_optimizer = optim.SGD(a_model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [ ]:
"""
Weight Initialization
"""
for m in a_model.modules():
    if isinstance(m, nn.Conv2d):
        ###todo

In [4]:
"""
Imports DataSets from pytorch db
"""
root = './data'
if not os.path.exists(root):
    os.mkdir(root)

data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.cat([x, x, x], 0))
])
#trans = transforms.ToTensor()    
mnist_trainset = dset.MNIST(root=root, train=True,transform = data_transform, download=True)
mnist_testset = dset.MNIST(root=root, train=False,transform = data_transform, download=True)

batch_size = 64

mnist_train_loader = torch.utils.data.DataLoader(
                 dataset=mnist_trainset,
                 batch_size=batch_size,
                 shuffle=True)
mnist_test_loader = torch.utils.data.DataLoader(
                dataset=mnist_testset,
                batch_size=batch_size,
                shuffle=False)

print('===>>> MNIST total training batch number: {}'.format(len(mnist_train_loader)))
print('===>>> MNIST total testing batch number: {}'.format(len(mnist_test_loader)))

===>>> MNIST total training batch number: 938
===>>> MNIST total testing batch number: 157


In [5]:
"""
Parameters
"""
epoches = 10

In [6]:
"""
Training
"""
a_model.train() #sets training mode
for epoch in range(epoches):
    ave_loss = 0
    for batch_idx, (x,target) in enumerate(mnist_train_loader):
        x = x.cuda()
        target = target.cuda()
        #set_trace()
        #print(x.shape)
        #x = torch.cat((x,x,x),0)
        
        out = a_model(x)
        loss = criterion(out,target)
        
        ave_loss = ave_loss * 0.9 + loss.item() * 0.1
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(batch_idx+1) % batch_size == 0 or (batch_idx+1) == len(mnist_train_loader):
            print('==>>> epoch: {}, batch index: {}, train loss {:.6f}'.format(epoch,batch_idx+1,ave_loss))

==>>> epoch: 0, batch index: 64, train loss 5.986807
==>>> epoch: 0, batch index: 128, train loss 2.465410
==>>> epoch: 0, batch index: 192, train loss 2.345332
==>>> epoch: 0, batch index: 256, train loss 2.353976
==>>> epoch: 0, batch index: 320, train loss 2.341865
==>>> epoch: 0, batch index: 384, train loss 2.344184
==>>> epoch: 0, batch index: 448, train loss 2.334264
==>>> epoch: 0, batch index: 512, train loss 2.330644
==>>> epoch: 0, batch index: 576, train loss 2.345826
==>>> epoch: 0, batch index: 640, train loss 2.323877
==>>> epoch: 0, batch index: 704, train loss 2.330319
==>>> epoch: 0, batch index: 768, train loss 2.331876
==>>> epoch: 0, batch index: 832, train loss 2.335341
==>>> epoch: 0, batch index: 896, train loss 2.324032
==>>> epoch: 0, batch index: 938, train loss 2.318449
==>>> epoch: 1, batch index: 64, train loss 2.319501
==>>> epoch: 1, batch index: 128, train loss 2.328300
==>>> epoch: 1, batch index: 192, train loss 2.329762
==>>> epoch: 1, batch index: 2

In [7]:
#torch.save(a_model, './AlexMnist')

In [49]:
"""
Testing
"""
a_model = a_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in mnist_test_loader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs = a_model(images)
        _, predicted = torch.max(outputs.data, 1)
        print(predicted)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()



tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
  

tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([ 27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27, 236,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27,  27,  27,  27,  27,  27,  27], device='cuda:0')
tensor([ 27, 228,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
         27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,  27,
    

tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
  

tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
  

tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
  

tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
        27, 27, 27, 27, 27, 27, 27, 27, 27, 27], device='cuda:0')
tensor([27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
  

In [9]:
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 97 %


In [11]:
#torch.save(a_model, './AlexMnist')

In [5]:
l = []
for k, layer in enumerate(a_model.modules()):
    if isinstance(layer, nn.Conv2d):
        l.append(layer)
        a = layer
        break
l

[Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))]

In [6]:
a

Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))

In [13]:
sum_dist = 0
for i,target in enumerate(a.weight):    
    sample = a.weight
    target = a.weight[i]
    sample = sample - target
    filter_distance = 0
    for it, k in enumerate(sample):
        dist = 0
        sub = k - target
        dist = torch.pow(sub,2)
        dist = torch.sum(dist)
        dist = torch.sqrt(dist)
        filter_distance += dist
        sum_dist += dist
#    print('Distance from target to filter {}'.format(it))
#    print(dist)
    print('For filter {}, total distance is {}'.format(i,filter_distance))
print('Total distance between filters: {}'.format(sum_dist))

For filter 0, total distance is 696.0435791015625
For filter 1, total distance is 684.5155639648438
For filter 2, total distance is 695.4371948242188
For filter 3, total distance is 696.2635498046875
For filter 4, total distance is 694.0437622070312
For filter 5, total distance is 692.6482543945312
For filter 6, total distance is 698.0197143554688
For filter 7, total distance is 689.8383178710938
For filter 8, total distance is 690.1146240234375
For filter 9, total distance is 692.4071044921875
For filter 10, total distance is 687.3634643554688
For filter 11, total distance is 693.4329223632812
For filter 12, total distance is 697.4518432617188
For filter 13, total distance is 693.4378051757812
For filter 14, total distance is 687.5142822265625
For filter 15, total distance is 694.6284790039062
For filter 16, total distance is 694.784423828125
For filter 17, total distance is 695.248779296875
For filter 18, total distance is 693.216796875
For filter 19, total distance is 693.7506103515

In [12]:
a_model.train()
kernel_optimizer.zero_grad()
    
sum_dist = 0
for i,target in enumerate(a.weight):
    sample = a.weight
    target = a.weight[i]
    sample = sample - target
    for it, k in enumerate(sample):
        dist = 0
        sub = k - target
        dist = torch.pow(sub,2)
        dist = torch.sum(dist)
        dist = torch.sqrt(dist)
        sum_dist += dist
filter_loss = - sum_dist
print(filter_loss)
filter_loss.backward()
kernel_optimizer.step()

tensor(-31298.2637, device='cuda:0', grad_fn=<NegBackward>)


In [36]:
a

63